# Analyzing CIA Factbook Data Using SQL

This project will be going over a cia factbook that holds data on every country and their attributes. I will be using MySQL to make insights based on the data shown. 

Now to import SQL into Jupyter notebook. 

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

### Opening up the tablesets and seeing what I'm working with

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE TYPE='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


I'm opening up the facts dataset because it has the data on the countries that I want.  

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Getting the min and max values for population and taking out the 'World' column

The whole world was taken into account so now I will run the same thing without the world value.

In [5]:
%%sql
SELECT MIN(population), MAX(population), 
       MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name <> 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


Okay now that we have gotten rid of the `'World'` row, the max population country seems to have 1.3 billion people. This is a realistic number. 

In [6]:
%%sql
SELECT ROUND(AVG(population), 2), ROUND(AVG(area), 2)
  FROM facts;

Done.


"ROUND(AVG(population), 2)","ROUND(AVG(area), 2)"
62094928.32,555093.55


### Finding countries with above average population with  below average area. 

Going to do this using a subquery(nested query) in the WHERE key phrase. 

In [7]:
%%sql
SELECT population AS pop, area, name
  FROM facts
 WHERE pop > (SELECT AVG(CAST(population AS FLOAT)) FROM facts)
   AND area < ROUND((SELECT AVG(CAST(area AS FLOAT)) FROM facts), 2);


Done.


pop,area,name
168957745,148460,Bangladesh
80854408,357022,Germany
126919659,377915,Japan
100998376,300000,Philippines
67976405,513120,Thailand
64088222,243610,United Kingdom
94348835,331210,Vietnam


### Creating a query that pulls 2 Country names that contain the largest population and the highest growth ratio. 

Going to accomplish this by using two subqueries(nested queires) in the SELECT key phrase. 

In [8]:
%%sql
SELECT
  (SELECT name FROM facts WHERE population = (SELECT MAX(population) FROM facts WHERE name <> 'World')) AS largest_population_name,
  (SELECT name FROM facts WHERE population_growth = (SELECT MAX(population_growth) FROM facts)) AS highest_growth_name
FROM facts

LIMIT 1;


Done.


largest_population_name,highest_growth_name
China,South Sudan


### Finding the countries with the highest ratio between land and water area 

In [9]:
%%sql
SELECT 
  name, 
  ROUND(CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT), 2) AS area_ratio
FROM 
  facts
  ORDER BY area_ratio DESC
  LIMIT 20;


Done.


name,area_ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


According to the chart above, British Indian Ocean Territory and The Virgin Islands have more water area, than land area. This is due to the antecedent being greater than the consequent. 

### Finding the top 3 countries with the highest potential for population growth. 

I will get this done by displaying the columns `name`, `birth_rate`, and `migration_rate`. The key phrase ORDER BY will add the birth_rate and migration_rate columns, then subtract the death_rate to make sure that were taking that into account. 

In [10]:
%%sql
SELECT name, birth_rate, migration_rate
  FROM facts
 ORDER BY (birth_rate + migration_rate) - death_rate DESC
 LIMIT 3;

Done.


name,birth_rate,migration_rate
South Sudan,36.91,11.47
American Samoa,22.89,21.13
Syria,22.17,19.79


The chart above illustrates the countries that look like they will be adding the most to their population next year. 

### Finding countries that have a higher death rate than birth rate.

I will be displaying the columns `name`, `birth_rate`, and `death_rate`. The `WHERE` key phrase will be used to display all of the coutries with a death rate higher than birth rate.  

In [11]:
%%sql
SELECT name, birth_rate, death_rate
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY birth_rate + death_rate DESC;

Done.


name,birth_rate,death_rate
Russia,11.6,13.69
Ukraine,10.72,14.46
Moldova,12.0,12.59
Lithuania,10.1,14.27
Latvia,10.0,14.31
Belarus,10.7,13.36
Bulgaria,8.92,14.44
Estonia,10.51,12.4
Serbia,9.08,13.66
Hungary,9.16,12.73


These are the countries that have a higher death rate than birth rate.

### Finding the area/population ratio
Going to find the area/population ratio by dividing the area column by the population column. Then, I'm going to set a conditional in the WHERE key phrase to skip over the `World` row. 

In [12]:
%%sql
SELECT name, ROUND(CAST(area as FLOAT) /CAST(population as FLOAT), 2) AS pop_area_ratio
  FROM facts
 WHERE name <> 'World'
 ORDER BY pop_area_ratio DESC
 LIMIT 10;
    

Done.


name,pop_area_ratio
Greenland,37.52
Svalbard,33.14
Falkland Islands (Islas Malvinas),3.62
Pitcairn Islands,0.98
Mongolia,0.52
Western Sahara,0.47
Namibia,0.37
Australia,0.34
Iceland,0.31
Guyana,0.29


It looks as if countries like Greenland and Svalbard have a very high population/area ratio. These countries have the most vacant space in the world. 

 

### Finding countries that have a higher death rate than birth rate. 

In [13]:
%%sql
SELECT name, ROUND(CAST(area as FLOAT) /CAST(population as FLOAT), 2) AS pop_area_ratio
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY pop_area_ratio DESC

Done.


name,pop_area_ratio
Russia,0.12
Estonia,0.04
Saint Pierre and Miquelon,0.04
Latvia,0.03
Belarus,0.02
Bulgaria,0.02
Lithuania,0.02
Austria,0.01
Bosnia and Herzegovina,0.01
Croatia,0.01


# Conclusion

In this SQL project, a comprehensive analysis of the CIA database has yielded valuable insights about various countries worldwide. Several key observations have been made:

* Population Growth: The project examined the population growth rates of different countries, providing a glimpse into their potential future populations.

* Population Challenges: Identification of countries facing population challenges, such as low birth rates or migration rates, was performed, shedding light on potential demographic issues.

* Water Dominant Countries: The analysis revealed countries where water area exceeded land area, indicating their unique geographical characteristics.

* High Population Density: By pinpointing countries with above-average populations and below-average land area, the project highlighted regions with high population density.

The findings from this project offer the CIA a foundational understanding of global demographics, enabling better-informed decisions and policies. By harnessing these insights, the CIA can gain valuable knowledge about the countries in their database, fostering a broader comprehension of the global landscape.